In [11]:
import pandas as pd
import numpy as np
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from data.helpers import constants, utils, vocabulary
from sklearn import linear_model

In [2]:
data = pickle.load(open("./data/data.p", "rb"))
vocab = pickle.load(open("./vocab.p", "rb"))

In [5]:
def dummy_fun(doc):
    return doc

tfidf = TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None,
    vocabulary=vocab.word_to_id,
    stop_words=vocab.unigram_counts.most_common(1000))  

In [8]:
df = data[(data.political_party=='r')|(data.political_party=='d')]

In [9]:
matrix = tfidf.fit_transform(df.tokens)

In [19]:
y = [1 if p == 'd' else 0 for p in df.political_party]

In [14]:
clf = linear_model.LogisticRegression()

In [20]:
clf.fit(matrix, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [38]:
democrat = np.argsort(clf.coef_, axis=None)[-11:-1]
republican = np.argsort(clf.coef_, axis=None)[0:10]

In [42]:
[vocab.id_to_word.get(i) for i in democrat]

['brandeis',
 'nor',
 'on',
 'where',
 'compare',
 'argued',
 '[DG]',
 'here',
 'cause',
 'mcreynolds']

In [43]:
[vocab.id_to_word.get(i) for i in republican]

['waite',
 'its',
 'wall.',
 'upon',
 '(2d)',
 'therefore,',
 'present',
 'holmes',
 'contra.mr.',
 'a']